# PointNet++ для семантической сегментации облака точек

Этот ноутбук настраивает окружение и запускает обучение модели PointNet++


## 1. Установка зависимостей


In [ ]:
# Установка всех необходимых библиотек
%pip install torch torchvision numpy scikit-learn tqdm matplotlib tensorboard -q


## 2. Клонирование репозитория или загрузка файлов


In [ ]:
# Клонировать репозиторий (данные уже включены!)
!git clone https://github.com/JohnSili/pointnet2-segmentation.git
%cd pointnet2-segmentation


## 3. Распаковка данных

**Данные уже включены в репозиторий!** Архив `data_for_colab.zip` содержит 500 PLY файлов.

Если нужно загрузить свои данные, используйте вариант B ниже.


In [ ]:
import zipfile
import os

# Вариант A: Распаковать данные из репозитория (уже включены)
if os.path.exists('data_for_colab.zip'):
    print("Распаковка данных из репозитория...")
    with zipfile.ZipFile('data_for_colab.zip', 'r') as zip_ref:
        zip_ref.extractall('.')
    print("✓ Данные распакованы из репозитория!")
else:
    print("⚠ Архив данных не найден в репозитории")

# Вариант B: Загрузить свои данные (раскомментируйте если нужно)
# from google.colab import files
# uploaded = files.upload()
# for filename in uploaded.keys():
#     if filename.endswith('.zip'):
#         with zipfile.ZipFile(filename, 'r') as zip_ref:
#             zip_ref.extractall('.')
#         print(f"✓ Данные {filename} распакованы!")

# Проверка наличия данных
data_dir = '3011-20251217T195928Z-1-001'
area = '3011'
if os.path.exists(os.path.join(data_dir, area)):
    ply_files = [f for f in os.listdir(os.path.join(data_dir, area)) if f.endswith('.ply')]
    print(f"✓ Найдено {len(ply_files)} PLY файлов")
else:
    print("⚠ Данные не найдены. Будем использовать синтетические данные.")
    data_dir = '.'
    area = 'synthetic'


## 4. Проверка данных


In [ ]:
from dataset import S3DISDataset
import torch
import os

# Автоматическое определение пути к данным
if os.path.exists('3011-20251217T195928Z-1-001/3011'):
    data_dir = '3011-20251217T195928Z-1-001'
    area = '3011'
    print("✓ Используем реальные данные из 3011-20251217T195928Z-1-001/3011")
else:
    data_dir = '.'
    area = 'synthetic'
    print("⚠ Реальные данные не найдены. Используем синтетические данные.")

# Проверка данных
dataset = S3DISDataset(data_dir, area=area, split='train', num_points=2048)
print(f'\nDataset size: {len(dataset)} файлов')
points, labels = dataset[0]
print(f'Points shape: {points.shape}, Labels shape: {labels.shape}')
print(f'Unique classes: {sorted(torch.unique(labels).tolist())}')

# Сохраняем для использования в обучении
DATA_DIR = data_dir
AREA = area


## 5. Обучение модели


In [ ]:
# Запуск обучения с вашими данными
# Если данные загружены, используйте:
#   --data_dir 3011-20251217T195928Z-1-001 --area 3011
# Если данных нет, используйте синтетические:
#   --data_dir . --area synthetic

!python train.py \
    --data_dir {DATA_DIR} \
    --area {AREA} \
    --num_points 2048 \
    --batch_size 8 \
    --epochs 50 \
    --lr 0.001 \
    --num_classes 13 \
    --device cuda \
    --save_dir ./checkpoints \
    --log_dir ./logs


## 6. Мониторинг обучения (TensorBoard)


In [ ]:
# Запуск TensorBoard
%load_ext tensorboard
%tensorboard --logdir ./logs


## 7. Тестирование модели


In [ ]:
!python test.py \
    --data_dir {DATA_DIR} \
    --area {AREA} \
    --checkpoint ./checkpoints/best_model.pth \
    --num_classes 13


## 8. Визуализация результатов


In [ ]:
!python visualize.py \
    --data_dir {DATA_DIR} \
    --area {AREA} \
    --checkpoint ./checkpoints/best_model.pth \
    --num_samples 5 \
    --output_dir ./visualizations


## 9. Скачивание результатов


In [ ]:
from google.colab import files
import zipfile
import os

# Создаем архив с результатами
with zipfile.ZipFile('results.zip', 'w') as zipf:
    if os.path.exists('checkpoints'):
        for root, dirs, files_list in os.walk('checkpoints'):
            for file in files_list:
                zipf.write(os.path.join(root, file))
    if os.path.exists('visualizations'):
        for root, dirs, files_list in os.walk('visualizations'):
            for file in files_list:
                zipf.write(os.path.join(root, file))

# Скачиваем архив
files.download('results.zip')
